In [96]:
import pandas as pd
import sklearn as sk
import cPickle as p
import dill
from sklearn.feature_extraction import DictVectorizer
from sklearn.cross_validation import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.grid_search import GridSearchCV
from sklearn import linear_model
import toolz
import ujson

def pick(whitelist, dicts):
    return [toolz.keyfilter(lambda k: k in whitelist, d)
            for d in dicts]

#define load the data loading method
def load_json_yelp(filepath):
    with open(filepath, 'rb') as f:
        data = f.readlines()
        data = map(lambda x: x.rstrip(), data)
        data_json_str = "[" + ','.join(data) + "]"
        data_df = pd.read_json(data_json_str)
    return data_df

#the place for class definition
class  Estimator(sk.base.BaseEstimator, sk.base.RegressorMixin):
    def __init__(self,model):
        self.est = model
        pass
    def fit(self,X,y):
        test_param = {'alpha':[0.01,0.05,0.1,0.3,0.5,1,10,100]}
        gscv = GridSearchCV(self.est, test_param)
        gscv.fit(X,y)
        self.est = gscv.best_estimator_
        #self.est.fit(X,y)
        return self
    
    def score(self,X,y):
        return self.est.score(X_test,y)
    
    def predict(self, X):
        return self.est.predict(X)

#here is the place for trnasformer
class Transformer(sk.base.BaseEstimator, sk.base.TransformerMixin):
    def __init__(self):
        self.col = 'attributes'
    
    def flatten_dict(self, Xdict, new_dict, namelist):
        p_dict = Xdict
        for key in p_dict.keys():
            if type(p_dict[key]) == dict:
                namelist.append(key)
                new_dict.update(self.flatten_dict(p_dict[key],new_dict,namelist))
                namelist.remove(key)
            elif type(p_dict[key]) in [unicode,str]:
                newkey = key + '_' + p_dict[key]
                for n in reversed(namelist):
                    newkey = n + '_' + newkey
                new_dict[newkey]=1
                #print newkey
            elif type(p_dict[key]) in [bool]:
                newkey = key
                for n in reversed(namelist):
                    newkey = n + '_' + newkey
                if p_dict[key]:
                    new_dict[newkey]=1
                else:
                    new_dict[newkey]=0
                
            elif type(p_dict[key]) not in [bool, int, float]:    
                raise ValueError("type error in flatten_dict!")
        return new_dict
    
    def fit(self,X,y=None):
        # flatten the train dict
        #new_dict = dict()
        #namelist = list()
        #attr_train = [self.flatten_dict(x['attributes'], new_dict, namelist) for x in X]
        #attr_train = [self.flatten_dict(x[self.col]) for x in X]
        # transform the training records
        #self.est.fit_transform(attr_train)
        attr_X = list()
        newX = X[self.col]
        new_dict = dict()
        namelist = list()
        for x in newX:
            attr_X.append(self.flatten_dict(x, new_dict, namelist))
        self.vectorizer = DictVectorizer()
        self.vectorizer.fit_transform(attr_X)
        return self
    
    def transform(self,X):
         # transform the test record
        newX = X[self.col]
        new_dict = dict()
        namelist = list()
        if type(X) is list:
            attr_X = [self.flatten_dict(record, new_dict, namelist) for record in newX]
        elif type(X) is dict:
            attr_X = self.flatten_dict(newX, new_dict, namelist)
            print attr_X
        else:
            attr_X = list()
            for x in newX:
                attr_X.append(self.flatten_dict(x, new_dict, namelist)) 
        X_trans = self.vectorizer.transform(attr_X)
        return X_trans 


#data loading
filepath = 'yelp_train.json'

print'start loading'
#data loading
filepath = 'yelp_train.json'
yelp = list()
with open(filepath, 'rb') as f:
    for line in f:
        yelp.append(ujson.loads(line))
df = pd.DataFrame.from_dict(yelp)
print 'done'
#true = True
#false = False
#testdic =  {"Take-out": true, "Wi-Fi": "no", "Good For": {"dessert": false, "latenight": false, "lunch": true, "dinner": false, "breakfast": false, "brunch": false}, "Caters": true, "Noise Level": "average", "Takes Reservations": false, "Delivery": false, "Ambience": {"romantic": false, "intimate": false, "classy": false, "hipster": false, "divey": false, "touristy": false, "trendy": false, "upscale": false, "casual": false}, "Parking": {"garage": false, "street": false, "validated": false, "lot": false, "valet": false}, "Has TV": true, "Outdoor Seating": true, "Attire": "casual", "Alcohol": "beer_and_wine", "Waiter Service": true, "Accepts Credit Cards": true, "Good for Kids": true, "Good For Groups": true, "Price Range": 2}
#t = Transformer()
#xt = t.transform(df)
#d = dict()
#l = list()
#a = t.flatten_dict(testdic,d,l)
#print a
#use the transformer
X = df
print type(X)
y = df['stars']
#the place for generating cross validation group
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
#tuned_variable = {'estimate__n_neighbors':[5,7,9,11,13,15]}
pipeline = Pipeline([('getcols',Transformer()),('estimate',Estimator(linear_model.Ridge()))])
print 'done'

start loading
done
<class 'pandas.core.frame.DataFrame'>
done


In [97]:
presult = pipeline.fit(X,y)

with open('attribute_model','wb') as output:
    dill.dump(presult,output)

with open("attribute_model", 'rb') as in_strm:
    datastruct = dill.load(in_strm)

print datastruct

presult = datastruct.predict(X)

print presult
print type(presult)

print 'done'

Pipeline(steps=[('getcols', Transformer()), ('estimate', Estimator(model=None))])
[ 3.6729137  3.6729137  3.6729137 ...,  3.6729137  3.6729137  3.6729137]
<type 'numpy.ndarray'>
done
